<a href="https://colab.research.google.com/github/Ankyytt/Hindi_Hate_Speech_Detection/blob/main/IndicBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# Definitive Advanced IndicBERT + LoRA Fine-tuning
#    (Manual PyTorch Training Loop - No Trainer API)
# =====================================================
# Features:
# - Bypasses the problematic Trainer/TrainingArguments API entirely.
# - Uses a standard, robust PyTorch manual training loop.
# - Handles multi-label classification for hate speech.
# - Uses LoRA for parameter-efficient fine-tuning.
# - Implements a weighted loss function to handle class imbalance.

# STEP 1: Run this command in your terminal or the first cell of your notebook
# !pip install -q transformers datasets peft accelerate evaluate bitsandbytes scikit-learn torch

import os
import pandas as pd
from datasets import Dataset # Corrected import
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate # Correct library for metrics

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
train_ds = train_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Training Setup
# =====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device) # Move weights to the correct device

# =====================================================
# 6️ Manual Training & Evaluation Loop
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        # Use weighted loss
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        loss = loss_fct(outputs.logits, batch["labels"].float())

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # --- Evaluation ---
    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
        labels = batch["labels"].cpu().numpy().astype(int)

        all_preds.extend(predictions)
        all_labels.extend(labels)

    # Compute F1 score
    f1_score = metric.compute(predictions=np.array(all_preds), references=np.array(all_labels), average="micro")["f1"]
    print(f"Epoch {epoch+1}/{num_epochs} | Validation F1 (micro): {f1_score:.4f}")

    # Save the best model
    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f" New best model saved with F1: {best_f1:.4f}")

# =====================================================
# 7️ Final Evaluation on Test Set
# =====================================================
print("\n Evaluating on the final test set with the best model...")
# Load the best model state
model.load_state_dict(torch.load(best_model_path))
model.eval()
all_preds_test = []
all_labels_test = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
    labels = batch["labels"].cpu().numpy().astype(int)

    all_preds_test.extend(predictions)
    all_labels_test.extend(labels)

test_f1 = metric.compute(predictions=np.array(all_preds_test), references=np.array(all_labels_test), average="micro")["f1"]

print("\n=====================================")
print(f"Final Test F1 Score (micro): {test_f1:.4f}")
print("=====================================")

ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
# =====================================================
# Definitive Advanced IndicBERT + LoRA Fine-tuning
#    (Manual PyTorch Training Loop - No Trainer API)
# =====================================================
# Features:
# - Bypasses the problematic Trainer/TrainingArguments API entirely.
# - Uses a standard, robust PyTorch manual training loop.
# - Handles multi-label classification for hate speech.
# - Uses LoRA for parameter-efficient fine-tuning.
# - Implements a weighted loss function to handle class imbalance.

# STEP 1: Run this command in your terminal or the first cell of your notebook
# !pip install -q transformers datasets peft accelerate evaluate bitsandbytes scikit-learn torch

import os
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
# Corrected imports for AdamW and get_scheduler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
train_ds = train_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Training Setup
# =====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device)

# =====================================================
# 6️ Manual Training & Evaluation Loop
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        loss = loss_fct(outputs.logits, batch["labels"].float())

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
        labels = batch["labels"].cpu().numpy().astype(int)

        all_preds.extend(predictions)
        all_labels.extend(labels)

    f1_score = metric.compute(predictions=np.array(all_preds), references=np.array(all_labels), average="micro")["f1"]
    print(f"Epoch {epoch+1}/{num_epochs} | Validation F1 (micro): {f1_score:.4f}")

    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f" New best model saved with F1: {best_f1:.4f}")

# =====================================================
# 7️ Final Evaluation on Test Set
# =====================================================
print("\n Evaluating on the final test set with the best model...")
model.load_state_dict(torch.load(best_model_path))
model.eval()
all_preds_test = []
all_labels_test = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
    labels = batch["labels"].cpu().numpy().astype(int)

    all_preds_test.extend(predictions)
    all_labels_test.extend(labels)

test_f1 = metric.compute(predictions=np.array(all_preds_test), references=np.array(all_labels_test), average="micro")["f1"]

print("\n=====================================")
print(f"Final Test F1 Score (micro): {test_f1:.4f}")
print("=====================================")

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

/tmp/ipython-input-2473776614.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 278,345,484 || trainable%: 0.1076


  0%|          | 0/2748 [00:00<?, ?it/s]

RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
# =====================================================
# Definitive Advanced IndicBERT + LoRA Fine-tuning
#    (Manual PyTorch Training Loop - No Trainer API)
# =====================================================
# Features:
# - Bypasses the problematic Trainer/TrainingArguments API entirely.
# - Uses a standard, robust PyTorch manual training loop.
# - Handles multi-label classification for hate speech.
# - Uses LoRA for parameter-efficient fine-tuning.
# - Implements a weighted loss function to handle class imbalance.

# STEP 1: Run this command in your terminal or the first cell of your notebook
# !pip install -q transformers datasets peft accelerate evaluate bitsandbytes scikit-learn torch

import os
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
# Corrected imports for AdamW and get_scheduler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
train_ds = train_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": [x[c] for c in label_cols]}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Training Setup
# =====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device)

# =====================================================
# 6️ Manual Training & Evaluation Loop
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        loss = loss_fct(outputs.logits, batch["labels"].float())

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
        labels = batch["labels"].cpu().numpy().astype(int)

        all_preds.extend(predictions)
        all_labels.extend(labels)

    f1_score = metric.compute(predictions=np.array(all_preds), references=np.array(all_labels), average="micro")["f1"]
    print(f"Epoch {epoch+1}/{num_epochs} | Validation F1 (micro): {f1_score:.4f}")

    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f" New best model saved with F1: {best_f1:.4f}")

# =====================================================
# 7️ Final Evaluation on Test Set
# =====================================================
print("\n Evaluating on the final test set with the best model...")
model.load_state_dict(torch.load(best_model_path))
model.eval()
all_preds_test = []
all_labels_test = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
    labels = batch["labels"].cpu().numpy().astype(int)

    all_preds_test.extend(predictions)
    all_labels_test.extend(labels)

test_f1 = metric.compute(predictions=np.array(all_preds_test), references=np.array(all_labels_test), average="micro")["f1"]

print("\n=====================================")
print(f"Final Test F1 Score (micro): {test_f1:.4f}")
print("=====================================")

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

/tmp/ipython-input-2473776614.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 278,345,484 || trainable%: 0.1076


  0%|          | 0/2748 [00:00<?, ?it/s]

RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
# =====================================================
# 🧠 Definitive Advanced IndicBERT + LoRA Fine-tuning
#    (Manual PyTorch Training Loop - No Trainer API)
# =====================================================
# Features:
# - Bypasses the problematic Trainer/TrainingArguments API entirely.
# - Uses a standard, robust PyTorch manual training loop.
# - Handles multi-label classification for hate speech.
# - Uses LoRA for parameter-efficient fine-tuning.
# - Implements a weighted loss function to handle class imbalance.

# STEP 1: Run this command in your terminal or the first cell of your notebook
# !pip install -q transformers datasets peft accelerate evaluate bitsandbytes scikit-learn torch

import os
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
# Ensure labels are float32 from the start
train_ds = train_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float32)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Training Setup
# =====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device)

# =====================================================
# 6️ Manual Training & Evaluation Loop (FINAL FIX)
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Separate inputs from labels
        labels = batch.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in batch.items()}

        # Get model outputs (logits)
        outputs = model(**inputs)

        # Calculate weighted loss
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        # Ensure labels are float32 for the loss function
        loss = loss_fct(outputs.logits, labels.to(torch.float32))

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        labels = batch.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)

        all_preds.extend(predictions)
        all_labels.extend(labels.cpu().numpy().astype(int))

    f1_score = metric.compute(predictions=np.array(all_preds), references=np.array(all_labels), average="micro")["f1"]
    print(f"Epoch {epoch+1}/{num_epochs} | Validation F1 (micro): {f1_score:.4f}")

    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ New best model saved with F1: {best_f1:.4f}")

# =====================================================
# 7️ Final Evaluation on Test Set
# =====================================================
print("\n🧪 Evaluating on the final test set with the best model...")
model.load_state_dict(torch.load(best_model_path))
model.eval()
all_preds_test = []
all_labels_test = []
for batch in test_dataloader:
    labels = batch.pop("labels")
    inputs = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)

    all_preds_test.extend(predictions)
    all_labels_test.extend(labels.numpy().astype(int))

test_f1 = metric.compute(predictions=np.array(all_preds_test), references=np.array(all_labels_test), average="micro")["f1"]

print("\n=====================================")
print(f"Final Test F1 Score (micro): {test_f1:.4f}")
print("=====================================")

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

/tmp/ipython-input-3384305152.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float32)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 278,345,484 || trainable%: 0.1076


  0%|          | 0/2748 [00:00<?, ?it/s]

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value('int32'), 'references': Value('int32')},
Input predictions: [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]],
Input references: [[0 0 0 1 0 0]
 [0 0 1 0 1 1]
 [0 0 1 0 1 1]
 ...
 [0 0 0 1 0 0]
 [0 0 0 0 0 1]
 [0 0 0 1 0 0]]

In [ ]:
# =====================================================
# Definitive Advanced IndicBERT + LoRA Fine-tuning
#    (Manual PyTorch Training Loop - No Trainer API)
# =====================================================
# Features:
# - Bypasses the problematic Trainer/TrainingArguments API entirely.
# - Uses a standard, robust PyTorch manual training loop.
# - Handles multi-label classification for hate speech.
# - Uses LoRA for parameter-efficient fine-tuning.
# - Implements a weighted loss function to handle class imbalance.

import os
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
train_ds = train_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float32)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Training Setup
# =====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device)

# =====================================================
# 6️ Manual Training & Evaluation Loop (FINAL FIX)
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        labels = batch.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        loss = loss_fct(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        labels = batch.pop("labels")
        inputs = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
        all_preds.extend(predictions)
        all_labels.extend(labels.numpy().astype(int))

    # FINAL FIX: Flatten the arrays to match the expected format for "micro" F1 score
    all_preds_flat = np.array(all_preds).flatten()
    all_labels_flat = np.array(all_labels).flatten()
    f1_score = metric.compute(predictions=all_preds_flat, references=all_labels_flat, average="micro")["f1"]

    print(f"\nEpoch {epoch+1}/{num_epochs} | Validation F1 (micro): {f1_score:.4f}")

    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with F1: {best_f1:.4f}")

# =====================================================
# 7️ Final Evaluation on Test Set
# =====================================================
print("\n Evaluating on the final test set with the best model...")
model.load_state_dict(torch.load(best_model_path))
model.eval()
all_preds_test = []
all_labels_test = []
for batch in test_dataloader:
    labels = batch.pop("labels")
    inputs = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
    all_preds_test.extend(predictions)
    all_labels_test.extend(labels.numpy().astype(int))

# Flatten the test arrays as well
all_preds_test_flat = np.array(all_preds_test).flatten()
all_labels_test_flat = np.array(all_labels_test).flatten()
test_f1 = metric.compute(predictions=all_preds_test_flat, references=all_labels_test_flat, average="micro")["f1"]

print("\n=====================================")
print(f" Final Test F1 Score (micro): {test_f1:.4f}")
print("=====================================")

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

/tmp/ipython-input-52674021.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float32)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 278,345,484 || trainable%: 0.1076


  0%|          | 0/2748 [00:00<?, ?it/s]


Epoch 1/3 | Validation F1 (micro): 0.7409
✅ New best model saved with F1: 0.7409

Epoch 2/3 | Validation F1 (micro): 0.7409

Epoch 3/3 | Validation F1 (micro): 0.7409

🧪 Evaluating on the final test set with the best model...

✅ Final Test F1 Score (micro): 0.7441


In [ ]:

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import torch
import numpy as np
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# =====================================================
# 1️ Load Data and Pre-trained Model (Setup)
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (1).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(base_model, peft_config)

# =====================================================
# 2️ Load Your Trained Weights
# =====================================================
best_model_path = "best_model.pth"
if not os.path.exists(best_model_path):
    print(f"Error: The model file '{best_model_path}' was not found.")
else:
    print(f"Loading trained weights from {best_model_path}...")
    model.load_state_dict(torch.load(best_model_path))
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    model.eval()

    # =====================================================
    # 3️ Make Predictions on the Test Set
    # =====================================================
    print("\nMaking predictions on the test set...")
    test_texts = test_df['text'].tolist()
    test_labels = test_df[mlb.classes_].values
    all_predictions = []

    with torch.no_grad():
        for text in tqdm(test_texts, desc="Predicting"):
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            preds = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
            all_predictions.append(preds[0])
    all_predictions = np.array(all_predictions)

    # =====================================================
    # 4️ Generate Classification Report & Show Errors
    # =====================================================
    print("\n Detailed Classification Report:\n")
    report = classification_report(test_labels, all_predictions, target_names=mlb.classes_, zero_division=0)
    print(report)

    predictions_df = test_df.copy()
    for i, label_name in enumerate(mlb.classes_):
        predictions_df[f'pred_{label_name}'] = all_predictions[:, i]

    misclassified_mask = (predictions_df[mlb.classes_].values != all_predictions).any(axis=1)
    misclassified_examples = predictions_df[misclassified_mask]

    print("\n\n Examples of Misclassified Text:\n")
    if misclassified_examples.empty:
        print("No misclassified examples found!")
    else:
        for index, row in misclassified_examples.head(5).iterrows():
            true_labels = [label for label in mlb.classes_ if row[label] == 1]
            predicted_labels = [label.replace('pred_', '') for label in predictions_df.columns if 'pred_' in label and row[label] == 1]

            print(f"--- Example ---")
            print(f"Text: '{row['text']}'")
            print(f"  - True Labels: {true_labels}")
            print(f"  - Predicted Labels: {predicted_labels}")
            print("-" * 20)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading trained weights from best_model.pth...

Making predictions on the test set...


Predicting:   0%|          | 0/1833 [00:00<?, ?it/s]


✅ Detailed Classification Report:

              precision    recall  f1-score   support

  defamation       0.00      0.00      0.00       485
        fake       0.00      0.00      0.00       175
        hate       0.00      0.00      0.00       566
    non-hate       0.00      0.00      0.00       781
    violence       0.00      0.00      0.00       392
      vulgar       0.00      0.00      0.00       415

   micro avg       0.00      0.00      0.00      2814
   macro avg       0.00      0.00      0.00      2814
weighted avg       0.00      0.00      0.00      2814
 samples avg       0.00      0.00      0.00      2814



✅ Examples of Misclassified Text:

--- Example ---
Text: 'अब बहुत हो गई पोर्किस में कि अब उड़ा देना चाहिए पाकिस्तान को'
  - True Labels: ['defamation', 'vulgar']
  - Predicted Labels: []
--------------------
--- Example ---
Text: 'Reliance-Future deal: फ्यूचर ग्रुप के शेयरहोल्डर्स को इस डील से क्या मिला, जानें 
https://t.co/pxgWo2Tymx via @NavbharatTimes 
#BizBaz

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import numpy as np

# --- Setup (run this once) ---
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
LABELS = ['defamation', 'fake', 'hate', 'non-hate', 'violence', 'vulgar'] # Ensure this order matches your training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load base model
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

# Apply LoRA config
peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
inference_model = get_peft_model(base_model, peft_config)

# Load your fine-tuned weights
inference_model.load_state_dict(torch.load("best_model.pth"))
inference_model.to(DEVICE)
inference_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# --- End of Setup ---


def predict_hate_speech(text: str):
    """Takes a Hindi text string and returns a list of predicted labels."""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(DEVICE)
        outputs = inference_model(**inputs)
        logits = outputs.logits

        # Get probabilities and apply threshold
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        predictions = (probabilities > 0.5).astype(int)

        # Map predictions back to label names
        predicted_labels = [LABELS[i] for i, pred in enumerate(predictions) if pred == 1]

    return predicted_labels

# --- Example Usage ---
new_text_1 = "यह एक बहुत ही अच्छी और सकारात्मक खबर है"
new_text_2 = "तुम जैसे लोगों को सबक सिखाना ही पड़ेगा"

print(f"Text: '{new_text_1}'\nPredicted Labels: {predict_hate_speech(new_text_1)}\n")
print(f"Text: '{new_text_2}'\nPredicted Labels: {predict_hate_speech(new_text_2)}\n")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: 'यह एक बहुत ही अच्छी और सकारात्मक खबर है'
Predicted Labels: []

Text: 'तुम जैसे लोगों को सबक सिखाना ही पड़ेगा'
Predicted Labels: []



In [ ]:
# =====================================================
# Final Inference Script for Custom Text
# =====================================================

# STEP 1: Install necessary libraries if in a new environment
# !pip install -q transformers datasets peft torch scikit-learn accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import numpy as np
import os

# =====================================================
# 1️ Setup the Model for Inference
# =====================================================
# --- Configuration ---
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
# This list of labels MUST be in the same order as it was during training
LABELS = ['defamation', 'fake', 'hate', 'non-hate', 'violence', 'vulgar']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BEST_MODEL_PATH = "best_model.pth"

# --- Load Model Architecture ---
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}

# Load the base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

# Apply the same LoRA configuration as in training
peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
inference_model = get_peft_model(base_model, peft_config)

# --- Load Your Fine-Tuned Weights ---
if not os.path.exists(BEST_MODEL_PATH):
    print(f"Error: The model file '{BEST_MODEL_PATH}' was not found. Please ensure it was saved correctly from the training script.")
else:
    inference_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval() # Set the model to evaluation mode

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # =====================================================
    # 2️ Create the Prediction Function
    # =====================================================
    def predict_category(text: str):
        """Takes a Hindi text string and returns its predicted labels."""
        if not text.strip():
            return ["No text provided"]

        with torch.no_grad():
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(DEVICE)

            # Get model predictions
            outputs = inference_model(**inputs)
            logits = outputs.logits

            # Convert logits to probabilities and apply a 0.5 threshold
            probabilities = torch.sigmoid(logits).cpu().numpy()[0]
            predictions = (probabilities > 0.5).astype(int)

            # Map the binary predictions back to label names
            predicted_labels = [LABELS[i] for i, pred in enumerate(predictions) if pred == 1]

            # If no label is predicted above the threshold, it's likely non-hate
            if not predicted_labels:
                return ["non-hate"]

        return predicted_labels

    # =====================================================
    # 3️ Run Predictions on Custom Text
    # =====================================================
    # Here you can add any text you want to test
    custom_texts_to_test = [
        "यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।", # Expected: non-hate
        "इस देश के गद्दारों को गोली मारो सालों को",         # Expected: hate, violence
        "तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?", # Expected: vulgar
        "यह खबर झूठी है, इसे विश्वास न करें।",              # Expected: fake
    ]

    print("Running predictions on custom text...\n")
    for i, text in enumerate(custom_texts_to_test):
        predicted_labels = predict_category(text)
        print(f"Input Text {i+1}: '{text}'")
        print(f"Predicted Categories: {predicted_labels}\n" + "-"*30)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Running predictions on custom text...

Input Text 1: 'यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।'
Predicted Categories: ['non-hate']
------------------------------
Input Text 2: 'इस देश के गद्दारों को गोली मारो सालों को'
Predicted Categories: ['non-hate']
------------------------------
Input Text 3: 'तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?'
Predicted Categories: ['non-hate']
------------------------------
Input Text 4: 'यह खबर झूठी है, इसे विश्वास न करें।'
Predicted Categories: ['non-hate']
------------------------------


In [ ]:
# =====================================================
# कस्टम टेक्स्ट के लिए अंतिम अनुमान स्क्रिप्ट
#    (ट्यून किए गए थ्रेसहोल्ड के साथ)
# =====================================================

# आवश्यक लाइब्रेरी इंस्टॉल करें
# !pip install -q transformers datasets peft torch scikit-learn accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import numpy as np
import os

# =====================================================
# 1️ अनुमान के लिए मॉडल सेटअप करें
# =====================================================
# --- कॉन्फ़िगरेशन ---
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
# यह लेबल सूची प्रशिक्षण के दौरान उपयोग की गई सूची के समान क्रम में होनी चाहिए
LABELS = ['defamation', 'fake', 'hate', 'non-hate', 'violence', 'vulgar']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BEST_MODEL_PATH = "best_model.pth"

# --- मॉडल आर्किटेक्चर लोड करें ---
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
inference_model = get_peft_model(base_model, peft_config)

# --- आपके फाइन-ट्यून किए गए वेट्स लोड करें ---
if not os.path.exists(BEST_MODEL_PATH):
    print(f"त्रुटि: मॉडल फ़ाइल '{BEST_MODEL_PATH}' नहीं मिली।")
else:
    inference_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval()

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # =====================================================
    # 2️ सुधारा हुआ भविष्यवाणी फ़ंक्शन बनाएं
    # =====================================================
    def predict_category_with_tuned_threshold(text: str, positive_threshold=0.3, non_hate_threshold=0.5):
        """एक हिंदी टेक्स्ट स्ट्रिंग लेता है और उसके अनुमानित लेबल लौटाता है।"""
        if not text.strip():
            return ["कोई टेक्स्ट नहीं दिया गया"]

        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(DEVICE)
            outputs = inference_model(**inputs)
            logits = outputs.logits

            probabilities = torch.sigmoid(logits).cpu().numpy()[0]

            predicted_labels = []
            for i, label in enumerate(LABELS):
                # 'non-hate' के लिए एक अलग (उच्च) थ्रेसहोल्ड का उपयोग करें
                if label == 'non-hate':
                    if probabilities[i] > non_hate_threshold:
                        predicted_labels.append(label)
                # अन्य सभी सकारात्मक लेबलों के लिए कम थ्रेसहोल्ड का उपयोग करें
                else:
                    if probabilities[i] > positive_threshold:
                        predicted_labels.append(label)

            # यदि केवल 'non-hate' का अनुमान लगाया गया है या कुछ भी अनुमान नहीं लगाया गया है, तो 'non-hate' लौटाएं
            if not predicted_labels or predicted_labels == ['non-hate']:
                return ["non-hate"]
            # यदि 'non-hate' के साथ अन्य लेबल भी हैं, तो केवल अन्य लेबल दिखाएं
            elif 'non-hate' in predicted_labels and len(predicted_labels) > 1:
                return [label for label in predicted_labels if label != 'non-hate']

        return predicted_labels

    # =====================================================
    # 3️ कस्टम टेक्स्ट पर भविष्यवाणी चलाएं
    # =====================================================
    custom_texts_to_test = [
        "यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।", # अपेक्षित: non-hate
        "इस देश के गद्दारों को गोली मारो सालों को",         # अपेक्षित: hate, violence
        "तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?", # अपेक्षित: vulgar
        "यह खबर झूठी है, इसे विश्वास न करें।",              # अपेक्षित: fake
    ]

    print(" कस्टम टेक्स्ट पर सुधारे हुए थ्रेसहोल्ड के साथ भविष्यवाणी...\n")
    for i, text in enumerate(custom_texts_to_test):
        predicted_labels = predict_category_with_tuned_threshold(text)
        print(f"इनपुट टेक्स्ट {i+1}: '{text}'")
        print(f"अनुमानित श्रेणियाँ: {predicted_labels}\n" + "-"*30)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ कस्टम टेक्स्ट पर सुधारे हुए थ्रेसहोल्ड के साथ भविष्यवाणी...

इनपुट टेक्स्ट 1: 'यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।'
अनुमानित श्रेणियाँ: ['non-hate']
------------------------------
इनपुट टेक्स्ट 2: 'इस देश के गद्दारों को गोली मारो सालों को'
अनुमानित श्रेणियाँ: ['non-hate']
------------------------------
इनपुट टेक्स्ट 3: 'तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?'
अनुमानित श्रेणियाँ: ['non-hate']
------------------------------
इनपुट टेक्स्ट 4: 'यह खबर झूठी है, इसे विश्वास न करें।'
अनुमानित श्रेणियाँ: ['non-hate']
------------------------------


In [ ]:
# =====================================================
#  Final Inference Script for Custom Text
#    (with Tuned Threshold)
# =====================================================

# Install necessary libraries if in a new environment
# !pip install -q transformers datasets peft torch scikit-learn accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import numpy as np
import os

# =====================================================
# 1️ Setup the Model for Inference
# =====================================================
# --- Configuration ---
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
# This list of labels MUST be in the same order as it was during training
LABELS = ['defamation', 'fake', 'hate', 'non-hate', 'violence', 'vulgar']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BEST_MODEL_PATH = "best_model.pth"

# --- Load Model Architecture ---
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
inference_model = get_peft_model(base_model, peft_config)

# --- Load Your Fine-Tuned Weights ---
if not os.path.exists(BEST_MODEL_PATH):
    print(f"Error: The model file '{BEST_MODEL_PATH}' was not found.")
else:
    inference_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval() # Set the model to evaluation mode

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # =====================================================
    # 2️ Create the Improved Prediction Function
    # =====================================================
    def predict_category_with_tuned_threshold(text: str, positive_threshold=0.3, non_hate_threshold=0.5):
        """Takes a Hindi text string and returns its predicted labels."""
        if not text.strip():
            return ["No text provided"]

        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(DEVICE)
            outputs = inference_model(**inputs)
            logits = outputs.logits

            probabilities = torch.sigmoid(logits).cpu().numpy()[0]

            predicted_labels = []
            for i, label in enumerate(LABELS):
                # Use a separate (higher) threshold for 'non-hate'
                if label == 'non-hate':
                    if probabilities[i] > non_hate_threshold:
                        predicted_labels.append(label)
                # Use a lower threshold for all other positive labels
                else:
                    if probabilities[i] > positive_threshold:
                        predicted_labels.append(label)

            # If only 'non-hate' is predicted or nothing is predicted, return 'non-hate'
            if not predicted_labels or predicted_labels == ['non-hate']:
                return ["non-hate"]
            # If 'non-hate' is predicted along with other labels, show only the other labels
            elif 'non-hate' in predicted_labels and len(predicted_labels) > 1:
                return [label for label in predicted_labels if label != 'non-hate']

        return predicted_labels

    # =====================================================
    # 3️ Run Predictions on Custom Text
    # =====================================================
    # You can add any Hindi text you want to test here
    custom_texts_to_test = [
        "यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।", # Expected: non-hate
        "इस देश के गद्दारों को गोली मारो सालों को",         # Expected: hate, violence
        "तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?", # Expected: vulgar
        "यह खबर झूठी है, इसे विश्वास न करें।",              # Expected: fake
    ]

    print(" Running predictions with tuned threshold on custom text...\n")
    for i, text in enumerate(custom_texts_to_test):
        predicted_labels = predict_category_with_tuned_threshold(text)
        print(f"Input Text {i+1}: '{text}'")
        print(f"Predicted Categories: {predicted_labels}\n" + "-"*30)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Running predictions with tuned threshold on custom text...

Input Text 1: 'यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।'
Predicted Categories: ['non-hate']
------------------------------
Input Text 2: 'इस देश के गद्दारों को गोली मारो सालों को'
Predicted Categories: ['non-hate']
------------------------------
Input Text 3: 'तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?'
Predicted Categories: ['non-hate']
------------------------------
Input Text 4: 'यह खबर झूठी है, इसे विश्वास न करें।'
Predicted Categories: ['non-hate']
------------------------------


In [ ]:
# =====================================================
# Definitive Debugging & Inference Script
# =====================================================

# STEP 1: Install necessary libraries if in a new environment
# !pip install -q transformers datasets peft torch scikit-learn accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import numpy as np
import os

# =====================================================
# 1️ Setup the Model for Inference
# =====================================================
# --- Configuration ---
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
LABELS = ['defamation', 'fake', 'hate', 'non-hate', 'violence', 'vulgar']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BEST_MODEL_PATH = "best_model.pth"

# --- Load Model Architecture ---
# THIS IS THE FIX: Added the missing label2id definition
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)} # <-- This line was missing

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
inference_model = get_peft_model(base_model, peft_config)

# --- Load Your Fine-Tuned Weights ---
if not os.path.exists(BEST_MODEL_PATH):
    print(f"Error: The model file '{BEST_MODEL_PATH}' was not found.")
else:
    inference_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval()

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # =====================================================
    # 2️ Create the Enhanced Prediction Function
    # =====================================================
    def diagnose_and_predict(text: str, positive_threshold=0.3):
        """Takes Hindi text, prints all probabilities, and returns the prediction."""
        if not text.strip():
            return "No text provided", {}

        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(DEVICE)
            outputs = inference_model(**inputs)
            logits = outputs.logits

            probabilities = torch.sigmoid(logits).cpu().numpy()[0]

            # --- This is the new debugging part ---
            all_probabilities = {label: f"{prob:.4f}" for label, prob in zip(LABELS, probabilities)}

            # --- Prediction logic with tuned threshold ---
            predicted_labels = []
            for i, label in enumerate(LABELS):
                # We ignore 'non-hate' for now and focus on positive classes
                if label == 'non-hate':
                    continue
                if probabilities[i] > positive_threshold:
                    predicted_labels.append(label)

            # If no positive class is detected, default to 'non-hate'
            if not predicted_labels:
                predicted_labels = ["non-hate"]

        return predicted_labels, all_probabilities

    # =====================================================
    # 3️ Run Predictions on Custom Text
    # =====================================================
    custom_texts_to_test = [
        "यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।",
        "इस देश के गद्दारों को गोली मारो सालों को",
        "तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?",
        "यह खबर झूठी है, इसे विश्वास न करें।",
    ]

    print("Running enhanced diagnostics and prediction...\n")
    for i, text in enumerate(custom_texts_to_test):
        predicted_labels, all_probs = diagnose_and_predict(text)
        print(f"Input Text {i+1}: '{text}'")
        print("--- Raw Probabilities ---")
        for label, prob in all_probs.items():
            print(f"- {label}: {prob}")
        print("---------------------------")
        print(f"Predicted Categories: {predicted_labels}\n" + "="*30 + "\n")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Running enhanced diagnostics and prediction...

Input Text 1: 'यह फिल्म तो बहुत अच्छी थी, सबने बढ़िया काम किया।'
--- Raw Probabilities ---
- defamation: 0.1719
- fake: 0.1550
- hate: 0.1882
- non-hate: 0.2402
- violence: 0.1665
- vulgar: 0.1582
---------------------------
Predicted Categories: ['non-hate']

Input Text 2: 'इस देश के गद्दारों को गोली मारो सालों को'
--- Raw Probabilities ---
- defamation: 0.1617
- fake: 0.1277
- hate: 0.1736
- non-hate: 0.2018
- violence: 0.1451
- vulgar: 0.1376
---------------------------
Predicted Categories: ['non-hate']

Input Text 3: 'तू जानता नहीं है मैं कौन हूँ, तेरी औकात क्या है?'
--- Raw Probabilities ---
- defamation: 0.2224
- fake: 0.1961
- hate: 0.2377
- non-hate: 0.2612
- violence: 0.2031
- vulgar: 0.2111
---------------------------
Predicted Categories: ['non-hate']

Input Text 4: 'यह खबर झूठी है, इसे विश्वास न करें।'
--- Raw Probabilities ---
- defamation: 0.1949
- fake: 0.1717
- hate: 0.2077
- non-hate: 0.2583
- violence: 0.1858
- vulgar

In [ ]:
# =====================================================
# FINAL Advanced IndicBERT + LoRA Training Script
#    (With Improved Training Strategy)
# =====================================================
# Features:
# - Increased epochs and adjusted learning rate for better learning.
# - Adds a learning rate scheduler with warmup to stabilize training.
# - Uses a robust manual PyTorch loop.
# - Implements a weighted loss function to handle class imbalance.

# STEP 1: Install necessary libraries
# !pip install -q transformers datasets peft accelerate evaluate bitsandbytes scikit-learn torch

import os
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import evaluate

# =====================================================
# 1️ Load & Prepare Dataset
# =====================================================
FILE_PATH = "hate_speech_hindi_cleaned (2).csv"
df = pd.read_csv(FILE_PATH)

df['label_list'] = df['label'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])
labels_df = pd.DataFrame(Y, columns=mlb.classes_)
df = df[['text']].join(labels_df)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
test_ds = Dataset.from_pandas(test_df, preserve_index=False)

# =====================================================
# 2️ Tokenize & Format Data
# =====================================================
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

label_cols = list(mlb.classes_)
train_ds = train_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
val_ds = val_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])
test_ds = test_ds.map(lambda x: {"labels": np.array([x[c] for c in label_cols], dtype=np.float32)}, remove_columns=label_cols+['text'])

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

# =====================================================
# 3️ Create DataLoaders & Class Weights
# =====================================================
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(val_ds, batch_size=16)
test_dataloader = DataLoader(test_ds, batch_size=16)

class_counts = train_df[label_cols].sum()
total_samples = len(train_df)
class_weights = torch.tensor(total_samples / (len(label_cols) * class_counts), dtype=torch.float32)

# =====================================================
# 4️ Load Model & Apply LoRA
# =====================================================
id2label = {i: label for i, label in enumerate(mlb.classes_)}
label2id = {label: i for i, label in enumerate(mlb.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(mlb.classes_), problem_type="multi_label_classification",
    id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

peft_config = LoraConfig(task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =====================================================
# 5️ Improved Training Setup
# =====================================================
# --- Hyperparameter Changes ---
LEARNING_RATE = 3e-5
NUM_EPOCHS = 5
# --- End of Changes ---

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
num_warmup_steps = int(0.1 * num_training_steps) # Use 10% of steps for warmup

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = class_weights.to(device)

# =====================================================
# 6️ Manual Training & Evaluation Loop
# =====================================================
progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("f1")
best_f1 = 0
best_model_path = "best_model.pth"

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch in train_dataloader:
        labels = batch.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
        loss = loss_fct(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        labels = batch.pop("labels")
        inputs = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions = (torch.sigmoid(logits) > 0.5).cpu().numpy().astype(int)
        all_preds.extend(predictions)
        all_labels.extend(labels.numpy().astype(int))

    all_preds_flat = np.array(all_preds).flatten()
    all_labels_flat = np.array(all_labels).flatten()
    f1_score = metric.compute(predictions=all_preds_flat, references=all_labels_flat, average="micro")["f1"]

    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS} | Validation F1 (micro): {f1_score:.4f}")

    if f1_score > best_f1:
        best_f1 = f1_score
        torch.save(model.state_dict(), best_model_path)
        print(f" New best model saved with F1: {best_f1:.4f}")

print("\n\n New model training complete! ")
print(f"The best model was saved to '{best_model_path}' with a validation F1 score of {best_f1:.4f}.")
print("You can now re-run the debugging/prediction script to test this new, improved model.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 278,345,484 || trainable%: 0.1076


  0%|          | 0/4580 [00:00<?, ?it/s]


Epoch 1/5 | Validation F1 (micro): 0.7737
✅ New best model saved with F1: 0.7737

Epoch 2/5 | Validation F1 (micro): 0.7753
✅ New best model saved with F1: 0.7753

Epoch 3/5 | Validation F1 (micro): 0.7756
✅ New best model saved with F1: 0.7756

Epoch 4/5 | Validation F1 (micro): 0.7759
✅ New best model saved with F1: 0.7759

Epoch 5/5 | Validation F1 (micro): 0.7759


✅✅✅ New model training complete! ✅✅✅
The best model was saved to 'best_model.pth' with a validation F1 score of 0.7759.
You can now re-run the debugging/prediction script to test this new, improved model.
